In [51]:
#%% Import all necessary libraries
import sys
sys.path.insert(0, '../modules')
import numpy as np
import dask.array as da
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 10})
from sklearn.decomposition import PCA
import time
import napari
from matmatrix import consistentPCA, hullAnalysis, EuAngfromN
import os.path
import glob
import pickle
import msd
from scipy import stats
from dask.array import from_zarr
from skimage import measure
import cv2 

# time settings in the light sheet
pxum = 0.115
camExposure_ms = 2
sweep_um = 15
stepsize_nm = 400
exp3D_ms = 1e-3 * camExposure_ms * (sweep_um*1e3/stepsize_nm) 
pxum = 0.115

In [49]:
# load raw images
this_file_dir = os.path.dirname(os.path.dirname(os.path.abspath("./")))
setName = "20220227_franky_MT_suc99_20um"
dataName = "timelapse_2022_02_28-03_03_30"
pathZarr = os.path.join(this_file_dir,
                        "DNA-Rotary-Motor", "Helical-nanotubes",
                        "Light-sheet-OPM", "Result-data",
                        setName, dataName, dataName+'-A.npy')
print(pathZarr)
stackRaw = np.array(da.from_npy_stack(pathZarr))

C:\Users\labuser\Dropbox (ASU)\Research\DNA-Rotary-Motor\Helical-nanotubes\Light-sheet-OPM\Result-data\20220227_franky_MT_suc99_20um\timelapse_2022_02_28-03_03_30\timelapse_2022_02_28-03_03_30-A.npy


In [239]:
# binarization, extract coordinates, and compute CM
blobBin = []
for frame in range(len(stackRaw)):
# for frame in range(58,60):

    # threshold image
    thresvalue = 0.15
    img0 = stackRaw[frame]
    img0 = img0/max(img0.ravel())
    img = img0>thresvalue
    
    # label each pixels for each clusters
    blobs = np.uint8(measure.label(img, background=0))
    
    # close holes
#     kernel = np.ones((6,6), np.uint8); 
#     blobs = cv2.morphologyEx(blobs, cv2.MORPH_CLOSE, kernel, iterations=1)

    # select largest
    if frame == 58:  # pick the largest for the 
        labels = np.unique(blobs.ravel())[1:] # spit out label numbers/tags
        sizes = np.array([np.argwhere(blobs==l).shape[0] for l in labels])
        keep = labels[np.argwhere((sizes == max(sizes)))[0]]
        blob = blobs == keep
        
        # constraint for the next frame with CM location
        X_f0 = np.argwhere(blob).astype('float')
        CM_f0 = np.array([sum(X_f0[:,j]) for j in range(X_f0.shape[1])])/X_f0.shape[0]
        
    else: # for the subsequent frame, pick the cluster closest to the previous frame
        labels = np.unique(blobs.ravel())[1:]
        CM_fN_diff = np.zeros(len(labels))
        CM_fN = np.zeros([len(labels), 3])
        for l in range(len(labels)): # goes through every clusters
            X_fN = np.argwhere(blobs==l+1)
            CM_fN[l] = np.array([sum(X_fN[:,j]) for j in range(X_fN.shape[1])])/X_fN.shape[0]
            CM_fN_diff[l] = np.linalg.norm(CM_f0-CM_fN[l])
        closest_idx = np.where(CM_fN_diff == min(CM_fN_diff))[0][0]
        CM_f0 = CM_fN[closest_idx] # update the constraint value
        keep = labels[closest_idx]
        blob = blobs == keep
    
    # store threshold/binarized image
    blobBin.append(blob)
    
    # extract coordinates
    X0 = np.argwhere(blob).astype('float') # coordinates 
    xb.append(X0) # store coordinates
    
    # compute CM
    CM1 = np.array([sum(X0[:,j]) for j in range(X0.shape[1])])/X0.shape[0]
    cm[frame,:] = CM1 # store center of mass
    
    # Use PCA to find the rotation matrix
    X = X0 - CM1 # shift all the coordinates into origin
    pca = PCA(n_components=3)
    pca.fit(X)
    axes = whEnd*pca.components_
    xb0.append(X)

    # Make the PCA consistent
    if frame == 0:
        axes_ref = axes
    else:
        axes, axes_ref = consistentPCA(axes, axes_ref)
    
# Convert to dask array
blobBin = da.from_array(blobBin)

In [240]:
# Check raw image & threshold
viewer = napari.Viewer(ndisplay=3)      
viewer.add_image(stackRaw, contrast_limits=[100,140],\
                    scale=[0.115,.115,.115],\
                    multiscale=False,colormap='gray',opacity=0.5)
viewer.add_image(blobBin, contrast_limits=[0,1],\
                    scale=[0.115,.115,.115],\
                    multiscale=False,colormap='green',opacity=0.5)
viewer.scale_bar.visible=True
viewer.scale_bar.unit='um'
viewer.scale_bar.position='top_right'
viewer.axes.visible = True
napari.run()